In [ ]:
import requests # to download html code
from bs4 import BeautifulSoup # to navigate through the html code
import pandas as pd
import numpy as np
import re  
from sklearn.preprocessing import StandardScaler
import pickle

In [ ]:
import sys
sys.path.insert(1,'/Users/devirughani/Desktop/IronHack/Week_6/Spotify_Midterm_Project/config.py')
from config import *

In [ ]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,
                                                           client_secret= client_secret_id))


In [ ]:
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
    except FileNotFoundError: 
        print("File not found!") 

In [ ]:

def song_recommender():
    '''
    1. Enter song
    2. Get audio features for that song with it's song id
    '''
    answer = True
    while (answer == True):
        print('Enter song name:')
        song = input() #Input asking for song name
    
        song_options = sp.search(q= song,limit=10)
    
        i=0
        for item in song_options['tracks']['items']:#for loop that displays song name, the artist and url 
            print(i,"I am thinking of this song: '{}' by '{}' and the url is: {}".format(item['name'],item['artists'][0]['name'], item['external_urls']['spotify']))
            i+=1
    
        print('. I am thinking of (Enter the song number):')
        print()
    
        song_selection  = input()#input which asks for the number of the song(ordered number)
        song_selection =int(song_selection)
   
        song_id = song_options['tracks']['items'][song_selection]['id']
    

        print("Browsing for songs")
        print()
    
        audio_feats = pd.DataFrame([sp.audio_features(song_id)[0]])
        selected_audio_feats=audio_feats[['danceability','energy','acousticness','instrumentalness','liveness','valence']]
    
        #Scaling audio features
        scaler = load("/Users/devirughani/Desktop/IronHack/Week_6/Spotify_Midterm_Project/scalers/kmeans_scaler.pickle")
    
        X_scaled = scaler.transform(selected_audio_feats)
        X_scaled_df = pd.DataFrame(X_scaled, columns = selected_audio_feats.columns)
    
        # Assigning cluster
        best_model = load("/Users/devirughani/Desktop/IronHack/Week_6/Spotify_Midterm_Project/models/Kmeans_models/model_pickles11.pickle")

        cluster = best_model.predict(X_scaled_df)
        #print(cluster)
    
        #Is song in hot 100? Else: find another song from not hot with same cluster 
        all_clustered_songs=pd.read_csv('/Users/devirughani/Desktop/IronHack/Week_6/Spotify_Midterm_Project/data/all_clustered_songs_final_with_id_df.csv')
    
        if (song_id in list(all_clustered_songs[all_clustered_songs['org'] == "Hot"]['id'].values)):
            song_recommendation = all_clustered_songs[(all_clustered_songs['org'] == "Hot") & (all_clustered_songs['cluster'] == cluster[0])].sample(n=1)
            #display(song_recommendation)
            print("Gnoosic recommends: ",song_recommendation['song_title'].values[0])
            print("by: ",song_recommendation['artist'].values[0])
            print()
    
    
        else:
            song_recommendation = all_clustered_songs[(all_clustered_songs['org'] == "Not Hot") & (all_clustered_songs['cluster'] == cluster[0])].sample(n=1)
            
            print("Gnoosic recommends: ",song_recommendation['song_title'].values[0])
            print("by: ",song_recommendation['artist'].values[0])
            rec_id=song_recommendation['id'].values[0]
            url = "https://open.spotify.com/track/" + rec_id
            print("The link to the song is: {}".format(url))
            print()
            
            qa_1 = input("Would you like another song recommendation based on the same song? [Yes = 1, No = 0]")
            print()
        
            if ((qa_1 == '1') or (qa_1 == "Yes") or (qa_1 == "yes")):
                new_song_recommendation = all_clustered_songs[(all_clustered_songs['org'] == "Hot") & (all_clustered_songs['cluster'] == cluster[0])].sample(n=1) 
                print("Gnoosic recommends: ",new_song_recommendation['song_title'].values[0])
                print("by: ",new_song_recommendation['artist'].values[0])
                rec_id2=new_song_recommendation['id'].values[0]
                url2 = "https://open.spotify.com/track/" + rec_id2
                print("The link to the song is: {}".format(url2))
                print()
    
            else:
                qa_2 = input("Would you like another song recommendation based on a different song? [Yes = 1, No = 0]")
                print()
                if ((qa_2 == '0') or (qa_2 == "no") or (qa_2 == "No")):
                    answer=False
                    print()
                    print("Thank you for using Gnoosic to explore new music!")
                    print()
                else:
                    answer = True
        print()

# song_recommender()

In [157]:
song_recommender()

Enter song name:
Thunderstruck
0 I am thinking of this song: 'Thunderstruck' by 'AC/DC' and the url is: https://open.spotify.com/track/57bgtoPSgt236HzfBOd8kj
1 I am thinking of this song: 'Thunderstruck' by 'AC/DC' and the url is: https://open.spotify.com/track/52UWtKlYjZO3dHoRlWuz9S
2 I am thinking of this song: 'Thunderstruck' by '2CELLOS' and the url is: https://open.spotify.com/track/1taz71WL8grTxW6JLiL1i8
3 I am thinking of this song: 'Thunderstruck - Violin Version' by 'Lisa Agnelli' and the url is: https://open.spotify.com/track/53G27kHLXQe71HwmN2evhU
4 I am thinking of this song: 'Thunderstruck' by 'The Wiggles' and the url is: https://open.spotify.com/track/0yxneHHpWRwap3NNtN65Re
5 I am thinking of this song: 'Thunderstruck - Live at River Plate Stadium, Buenos Aires, Argentina - December 2009' by 'AC/DC' and the url is: https://open.spotify.com/track/7eHlRQ5acXEIMdGAKe00fO
6 I am thinking of this song: 'Thunderstruck' by 'Owl City' and the url is: https://open.spotify.com/tra

KeyboardInterrupt: Interrupted by user

In [159]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [ ]:
#return print(color.BOLD +"You should try: "+ color.END)

In [160]:
song_recommender()

Enter song name:
Love me Tender
0 I am thinking of this song: 'Love Me Tender' by 'Elvis Presley' and the url is: https://open.spotify.com/track/7iBBcw61QVJxI3NDzlpX2E
1 I am thinking of this song: 'Passin' Me By' by 'The Pharcyde' and the url is: https://open.spotify.com/track/4G3dZN9o3o2X4VKwt4CLts
2 I am thinking of this song: 'Love Me Tender - with The Royal Philharmonic Orchestra' by 'Elvis Presley' and the url is: https://open.spotify.com/track/1bZdOnI0GIcB2xqQqQH7DY
3 I am thinking of this song: 'Black Velvet' by 'Alannah Myles' and the url is: https://open.spotify.com/track/1KU5EHSz04JhGg3rReGJ0N
4 I am thinking of this song: 'Love Me Tenderly' by 'The Felice Brothers' and the url is: https://open.spotify.com/track/2BdBO8ei2fuIqScUjHRki1
5 I am thinking of this song: 'Emotions' by 'Mariah Carey' and the url is: https://open.spotify.com/track/0cELvuwJW1acISUHYB6suj
6 I am thinking of this song: 'Love Me Tender - From the 20th Century-Fox CinemaScope Production, "Love Me Tender"'